# pf2 to csv converter

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# read pf2 file
def read_pf2(fname):
    f = open(fname, 'r')
    lns = f.readlines()
    f.close()
    
    # get metadata stuff
    meta = {}
    start_idx = None
    
    for i, ln in enumerate(lns):
        ln = ln.strip()
        
        if ':' in ln and not ln.startswith('Data:'):
            pts = ln.split(':', 1)
            if len(pts) == 2:
                k = pts[0].strip()
                v = pts[1].strip()
                meta[k] = v
        
        # find where data starts
        if ln.startswith('Data:'):
            start_idx = i + 1
            break
    
    # read data lines
    data_lns = [ln.strip() for ln in lns[start_idx:] if ln.strip()]
    
    # parse it
    dat = []
    for ln in data_lns:
        vals = [x.strip() for x in ln.split(',')]
        dat.append(vals)
    
    # make dataframe
    df = pd.DataFrame(dat, columns=['t', 'alt', 'vel', 'temp', 'volt'])
    df = df.apply(pd.to_numeric, errors='coerce')
    
    return meta, df

In [3]:
# save to csv
def save_csv(d, m, outfile):
    f = open(outfile, 'w')
    f.write("# pnut data\n")
    for k, v in m.items():
        f.write(f"# {k}: {v}\n")
    f.write("#\n")
    f.close()
    
    d.to_csv(outfile, mode='a', index=False)
    print(f"saved {outfile}")

In [ ]:
# put your file here
infile = 'Laucnh_38.pf2'

# read it
m, d = read_pf2(infile)

# show metadata
print("flight info:")
for k, v in m.items():
    print(f"{k}: {v}")

print("\ndata:")
print(d.head(10))
print(f"\ntotal rows: {len(d)}")

In [ ]:
# save to csv
outfile = infile.replace('.pf2', '.csv')
save_csv(d, m, outfile)

In [ ]:
# plot stuff
fig, ax = plt.subplots(2, 2, figsize=(12, 8))

# altitude
ax[0, 0].plot(d['t'], d['alt'])
ax[0, 0].set_xlabel('time (s)')
ax[0, 0].set_ylabel('altitude (ft)')
ax[0, 0].set_title('altitude')
ax[0, 0].grid(True)

# velocity
ax[0, 1].plot(d['t'], d['vel'])
ax[0, 1].set_xlabel('time (s)')
ax[0, 1].set_ylabel('velocity (ft/s)')
ax[0, 1].set_title('velocity')
ax[0, 1].grid(True)

# temp
ax[1, 0].plot(d['t'], d['temp'])
ax[1, 0].set_xlabel('time (s)')
ax[1, 0].set_ylabel('temp (F)')
ax[1, 0].set_title('temperature')
ax[1, 0].grid(True)

# voltage
ax[1, 1].plot(d['t'], d['volt'])
ax[1, 1].set_xlabel('time (s)')
ax[1, 1].set_ylabel('voltage (V)')
ax[1, 1].set_title('battery')
ax[1, 1].grid(True)

plt.tight_layout()

# save plot as image
plotname = infile.replace('.pf2', '_plots.png')
plt.savefig(plotname, dpi=150, bbox_inches='tight')
print(f"saved plot to {plotname}")

plt.show()

In [ ]:
# stats
max_alt = d['alt'].max()
max_alt_t = d.loc[d['alt'].idxmax(), 't']
max_v = d['vel'].max()
min_v = d['vel'].min()
tot_t = d['t'].max()
avg_temp = d['temp'].mean()
avg_volt = d['volt'].mean()

print("stats:")
print(f"max altitude: {max_alt:.1f} ft")
print(f"time to apogee: {max_alt_t:.2f} s")
print(f"max velocity: {max_v:.1f} ft/s")
print(f"max descent: {abs(min_v):.1f} ft/s")
print(f"flight time: {tot_t:.2f} s")
print(f"avg temp: {avg_temp:.2f} F")
print(f"avg voltage: {avg_volt:.2f} V")

# save stats to file
statsfile = infile.replace('.pf2', '_stats.txt')
with open(statsfile, 'w') as f:
    f.write(f"Flight Stats for {infile}\n")
    f.write("=" * 40 + "\n")
    f.write(f"max altitude: {max_alt:.1f} ft\n")
    f.write(f"time to apogee: {max_alt_t:.2f} s\n")
    f.write(f"max velocity: {max_v:.1f} ft/s\n")
    f.write(f"max descent: {abs(min_v):.1f} ft/s\n")
    f.write(f"flight time: {tot_t:.2f} s\n")
    f.write(f"avg temp: {avg_temp:.2f} F\n")
    f.write(f"avg voltage: {avg_volt:.2f} V\n")
print(f"\nsaved stats to {statsfile}")

In [5]:
# batch convert - put all pf2 files in same folder
dir = '/Users/poorvivijay/Downloads/Altimeter 1'

pf2s = [f for f in os.listdir(dir) if f.endswith('.pf2')]
print(f"found {len(pf2s)} files\n")

for pf in pf2s:
    fp = os.path.join(dir, pf)
    
    try:
        # read file
        m, d = read_pf2(fp)
        
        # save csv
        out = fp.replace('.pf2', '.csv')
        save_csv(d, m, out)
        
        # make plots
        fig, ax = plt.subplots(2, 2, figsize=(12, 8))
        
        ax[0, 0].plot(d['t'], d['alt'])
        ax[0, 0].set_xlabel('time (s)')
        ax[0, 0].set_ylabel('altitude (ft)')
        ax[0, 0].set_title('altitude')
        ax[0, 0].grid(True)
        
        ax[0, 1].plot(d['t'], d['vel'])
        ax[0, 1].set_xlabel('time (s)')
        ax[0, 1].set_ylabel('velocity (ft/s)')
        ax[0, 1].set_title('velocity')
        ax[0, 1].grid(True)
        
        ax[1, 0].plot(d['t'], d['temp'])
        ax[1, 0].set_xlabel('time (s)')
        ax[1, 0].set_ylabel('temp (F)')
        ax[1, 0].set_title('temperature')
        ax[1, 0].grid(True)
        
        ax[1, 1].plot(d['t'], d['volt'])
        ax[1, 1].set_xlabel('time (s)')
        ax[1, 1].set_ylabel('voltage (V)')
        ax[1, 1].set_title('battery')
        ax[1, 1].grid(True)
        
        plt.tight_layout()
        
        # save plot
        plotname = fp.replace('.pf2', '_plots.png')
        plt.savefig(plotname, dpi=150, bbox_inches='tight')
        plt.close()
        
        # calc stats
        max_alt = d['alt'].max()
        max_alt_t = d.loc[d['alt'].idxmax(), 't']
        max_v = d['vel'].max()
        min_v = d['vel'].min()
        tot_t = d['t'].max()
        avg_temp = d['temp'].mean()
        avg_volt = d['volt'].mean()
        
        # save stats
        statsfile = fp.replace('.pf2', '_stats.txt')
        with open(statsfile, 'w') as sf:
            sf.write(f"Flight Stats for {pf}\n")
            sf.write("=" * 40 + "\n")
            sf.write(f"max altitude: {max_alt:.1f} ft\n")
            sf.write(f"time to apogee: {max_alt_t:.2f} s\n")
            sf.write(f"max velocity: {max_v:.1f} ft/s\n")
            sf.write(f"max descent: {abs(min_v):.1f} ft/s\n")
            sf.write(f"flight time: {tot_t:.2f} s\n")
            sf.write(f"avg temp: {avg_temp:.2f} F\n")
            sf.write(f"avg voltage: {avg_volt:.2f} V\n")
        
        print(f"  flight {m.get('Flight Number', '?')}: {m.get('Apogee', '?')} - saved csv, plot, stats")
        
    except Exception as e:
        print(f"  error on {pf}: {e}")

print("\ndone")

found 6 files

saved /Users/poorvivijay/Downloads/Altimeter 1/Launch 39.csv
  flight 39: 839' AGL - saved csv, plot, stats
saved /Users/poorvivijay/Downloads/Altimeter 1/Launch 40.csv
  flight 40: 784' AGL - saved csv, plot, stats
saved /Users/poorvivijay/Downloads/Altimeter 1/Launch 41.csv
  flight 41: 763' AGL - saved csv, plot, stats
saved /Users/poorvivijay/Downloads/Altimeter 1/Laucnh 38.csv
  flight 38: 662' AGL - saved csv, plot, stats
saved /Users/poorvivijay/Downloads/Altimeter 1/Launch 42.csv
  flight 42: 789' AGL - saved csv, plot, stats
saved /Users/poorvivijay/Downloads/Altimeter 1/Launch 37.csv
  flight 38: 662' AGL - saved csv, plot, stats

done
